# Multiple Linear Regression

### Objectives
After completing this lab, you will be able to:
- Use scikit-learn to implement multiple linear regression
- Create, train, and test a multiple linear regression model on real data

# Import Library

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

# Load the data 

In [2]:
url= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%202/data/FuelConsumptionCo2.csv"

In [3]:
df = pd.read_csv(url)
df.sample(5)

,MODELYEAR,MAKE,MODEL,VEHICLECLASS,ENGINESIZE,CYLINDERS,TRANSMISSION,FUELTYPE,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS
646,2014,LAND ROVER,RANGE ROVER SPORT V8 5.0 SC,SUV - STANDARD,5.0,8,AS8,Z,17.3,12.2,15.0,19,345
606,2014,KIA,FORTE 5,FULL-SIZE,1.6,4,M6,X,11.1,8.2,9.8,29,225
322,2014,DODGE,DART TURBO AERO,MID-SIZE,1.4,4,M6,X,8.5,5.8,7.3,39,168
1016,2014,VOLKSWAGEN,BEETLE,COMPACT,2.0,4,A6,Z,10.3,7.7,9.1,31,209
324,2014,DODGE,DURANGO AWD FFV,SUV - STANDARD,3.6,6,A8,X,13.9,9.8,12.1,23,278


### FuelConsumption.csv:
##### You will download and use a fuel consumption dataset, FuelConsumption.csv, which contains model-specific fuel consumption ratings and estimated carbon dioxide emissions for new light-duty vehicles for retail sale in Canada. Dataset source

- MODEL YEAR e.g. 2014
- MAKE e.g. VOLVO
- MODEL e.g. S60 AWD
- VEHICLE CLASS e.g. COMPACT
- ENGINE SIZE e.g. 3.0
- CYLINDERS e.g 6
- TRANSMISSION e.g. AS6
- FUEL TYPE e.g. Z
- FUEL CONSUMPTION in CITY(L/100 km) e.g. 13.2
- FUEL CONSUMPTION in HWY (L/100 km) e.g. 9.5
- FUEL CONSUMPTION COMBINED (L/100 km) e.g. 11.5
- FUEL CONSUMPTION COMBINED MPG (MPG) e.g. 25
- CO2 EMISSIONS (g/km) e.g. 182
#### Your task will be to create a multiple linear regression model using some of these features to predict CO2 emissions of unobserved cars based on the selected features.